In [1]:
import os
import pathlib
import shutil
import json
import polars as pl
from pprint import pprint
from tqdm import tqdm

pl.Config.load_from_file("pl_config.json")
pl.set_random_seed(999)

## 다운로드

다운받을 데이터셋 정보 조회

In [2]:
!aihubshell -mode l -datasetkey 624

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─030.웹데이터 기반 한국어 말뭉치 데이터
        ├─01.데이터
        │  ├─1.Training
        │  │  ├─라벨링데이터
        │  │  │  └─TL1.zip | 4 GB | 32785
        │  │  └─원천데이터
        │  │      └─TS1.zip | 4 GB | 32786
        │  └─2.Validation
        │      ├─라벨링데이터
        │      │  └─VL1.zip | 550 MB | 32787
        │      └─원천데이터
        │          └─VS1.zip | 472 MB | 32788
        ├─02.저작도구
        │  └─저작도구 사용매뉴얼.zip | 14 MB | 54504
        └─03.AI 모델
            └─AI모델 및 소스코드.zip | 811 MB | 487494


구어체 데이터는 원천 데이터, 라벨링 데이터로 구성되어 있다. AI Hub 데이터셋 소개란에 기재된 [개요](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=624)에 따르면:

* 원천 데이터
  * 원시데이터(웹데이터)로부터 비문, 비속어, 편향성, 비식별화 등 정제된 JSON 구조의 텍스트 데이터 10억 어절 이상
* 라벨링 데이터
  * 원천데이터로부터 JSON구조의 구문에 맞게 개체명과 신조어, 형태소(간투어, 부사 등)에 대하여 태깅된 텍스트 데이터 10억 어절 이상

따라서, 언어모델 사전학습용 말뭉치 구성을 위한 텍스트만 추출해야 한다면 원천데이터(32786, 32788)만 사용하면 된다.

In [3]:
archive_dir = [path for path in pathlib.Path(os.getcwd()).glob("030*") if path.suffix != ".ipynb"][0]
result_dir = pathlib.Path(f"{os.getcwd()}/aihub-datasets")
result_dir = result_dir.joinpath(archive_dir.name)
if result_dir.exists():
    os.system(f"rm -rf {result_dir}")
result_dir.mkdir(parents=True, exist_ok=True)

## 압축 해제 및 EDA

원천데이터 압축파일 경로 정의

In [4]:
archive_paths = []
for walk in os.walk(archive_dir):
    current_path, folders, files = walk
    for file_name in files:
        if file_name.endswith(".zip") and current_path.endswith("원천데이터"):
            archive_paths.append(f"{current_path}/{file_name}")

압축 해제

In [5]:
for archive_path in archive_paths:
    data_dir = pathlib.Path(archive_path.strip(".zip"))
    shutil.unpack_archive(
        filename=archive_path,
        extract_dir=data_dir,
        format="zip",
    )

각 데이터는 json 형태로 저장되어 있다. 파일 하나가 게시글 하나고, 하위 필드에 댓글 등 구어체 데이터가 달려있는 양상을 보인다.

In [6]:
data_files = list(archive_dir.joinpath("01.데이터").glob("*/원천데이터/[TV]S1/*/*.json"))
print(f"Number of files in dataset : {len(data_files):,}")

Number of files in dataset : 58,997


데이터 속성값들. 여기서도 `(이름)` 태그가 사용되어 비식별화가 적용된 것을 확인할 수 있다.

In [7]:
with open(data_files[0], "r") as file:
    documents = json.load(file)["SJML"]["text"]
pprint(documents[:4])

[{'board': '스포츠_해외축구',
  'content': '독일 도르트문트의 공격수 엘링 홀란드(20)의 활약에 그의 에이전트 미노 라이올라(53)가 활짝 웃었다.. . '
             "라이올라는 7일(한국시간) 영국 미러를 통해 '홀란드의 도르트문트 이적은 정말 잘 한 결정이다'고 웃었다.. . "
             '1월 이적 시장을 통해 도르트문트 유니폼을 입은 홀란드는 데뷔전이었던 지난달 18일 아우크스부르크전부터 해트트릭을 '
             '터뜨려 주목을 받았고, 25일 쾰른과 홈경기에서도 멀티골을 기록했다. 지난 1일 우니온 베를린전에서도 2골을 '
             '터뜨렸다. 그리고 지난 5일 독일축구협회(dfb)컵 16강 베르더 브레멘전에서 1골을 추가했다. 도르트문트 이적 후 '
             '4경기에서 무려 8골을 뽑아낸 것이다. 그야말로 엄청난 결정력이다.. . 홀란드는 사실 맨유행이 유력했다. 하지만 '
             '맨유는 홀란드의 에이전트 라이올라와의 사이가 갑작스럽게 틀어졌다. 라이올라는 홀란드의 아버지와 맨유 훈련장까지 '
             '방문했지만 끝내 결렬됐다. 현지 보도에 따르면 라이올라가 무리한 수수료를 요구한 것으로 전해졌다. 홀란드는 '
             "도르트문트로 이적했고, 승승장구하고 있다.. . 라이올라는 '지난 1년간 홀란드, 그의 아버지와 바쁜 나날을 "
             "보냈다. 많은 클럽과 이야기하며 그들의 계획을 듣고, 보고, 협상했다'며 '홀란드가 좋은 결정을 할 수 있게 "
             "도와줬다'고 만족감을 드러냈다..",
  'source_site': '스타뉴스',
  'subtitle': '',
  'title': "'이적 후 4g 8골' 홀란드 활약에 에이전트 라이올라 '잘한 결정이었지'",
  'url': 'https://star.mt.co.kr/stview.php?no=20200207154538

전체 로드

* 글자수가 10 이상인 문장들만 사용
* 텍스트의 첫문장과 끝문장이 서명인 경우가 다수 있어서 처음과 끝 두 문장은 날림
* 전체 데이터셋 용량이 꽤 크므로 파티션으로 분할

In [8]:
corpus = []
partition_idx = 0
for data_file in tqdm(data_files):
    with open(data_file, "r") as file:
        documents = json.load(file)["SJML"]["text"]
    for document in documents:
        sentences = filter(lambda x: len(x) >= 10 and "(이름)" not in x, document["content"].split(". ")[1:-1])
        sentences = map(lambda x: x.replace(".", ""), sentences)
        corpus.extend(list(sentences))
    if len(corpus) >= 2_000_000:
        partition = pl.DataFrame({"document": corpus})
        partition.write_parquet(result_dir.joinpath(f"partition_{partition_idx:02}.parquet"))
        partition_idx += 1
        corpus = []
if corpus:
    partition = pl.DataFrame({"document": corpus})
    partition.write_parquet(result_dir.joinpath(f"partition_{partition_idx:02}.parquet"))

100%|████████████████████████████████████| 58997/58997 [02:05<00:00, 471.53it/s]


### 문서 토큰 수 분포 확인

한국어에서는 정확히 적용되지 않지만, 편의를 위해 어절의 수(`공백 수 + 1`)를 토큰 수라고 정의한다. 6억개 가량의 토큰을 획득할 수 있는 corpus임을 알 수 있다.

In [11]:
num_tokens = []
for partition_path in tqdm(result_dir.glob("*.parquet")):
    partition = pl.read_parquet(partition_path)
    document_tokens = (
        partition
        .with_columns((pl.col("document").str.count_matches(" ") + 1).alias("num_tokens"))
        .to_dict(as_series=False)["num_tokens"]
    )
    num_tokens.extend(document_tokens)
print(f"Number of tokens in the corpus : {sum(num_tokens):,}")

20it [00:18,  1.06it/s]

Number of tokens in the corpus : 592,263,094


문서당 토큰 수 분포

In [12]:
pl.DataFrame({"num_tokens": num_tokens}).select(
    pl.col("num_tokens").min().alias("min"),
    pl.col("num_tokens").quantile(0.05).cast(pl.Int32).alias("q05"),
    pl.col("num_tokens").quantile(0.25).cast(pl.Int32).alias("Q1"),
    pl.col("num_tokens").quantile(0.50).cast(pl.Int32).alias("median"),
    pl.col("num_tokens").quantile(0.75).cast(pl.Int32).alias("Q3"),
    pl.col("num_tokens").quantile(0.95).cast(pl.Int32).alias("q95"),
    pl.col("num_tokens").max().alias("max"),
)

min,q05,Q1,median,Q3,q95,max
i64,i32,i32,i32,i32,i32,i64
1,4,9,13,19,32,"2,637"
